# imports

In [1]:
import numpy as np
import cv2
from shapely.geometry import Polygon, box

# Preprocessing the data
before the detections can take place the raw video must be preprocessed 

In [2]:
'''the auxillary function responsible  for alligning the video frame to the reference frame to infer the coordinates
of the grid'''
def show_matches(bf, kp1, desc1, kp2, desc2,reference_pick,frame ,num_matches=30):
    matches = bf.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    pts_ref = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts_frame = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, _ = cv2.findHomography(pts_frame, pts_ref, cv2.RANSAC, 5.0)
    aligned_frame = cv2.warpPerspective(frame, H, (reference_pick.shape[1], reference_pick.shape[0]))
    
    

    matches = cv2.drawMatches(
        reference_pick,
        kp1,
        frame,
        kp2,
        matches[:num_matches],
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    )
    return H, aligned_frame

'''usefull function to get the frid tiles coordinates forom the stored annotations - annotations were stored in format 
such that for each polygon 4 corners stored and for circle the center and 4 radii in perpendicular directions'''
def draw_polygons(image, annotations, color=(0, 255, 0)):
    for key, polygons in annotations.items():
        for polygon in polygons:
            pts = np.array(polygon, dtype=np.int32)
            if len(pts) ==4:
                cv2.polylines(image, [pts], isClosed=True, color=color, thickness=2)
            elif len(pts) ==5:
                center_coordinates = (pts[0][0],pts[0][1])
                rads = pts[1:]-pts[0]
                rads = np.abs(rads)
                rads = np.max(rads,axis=1)
                rads = np.mean(rads)
                radius=int(rads)
                cv2.circle(image, center_coordinates, radius, color, thickness=2)


'''function used in order to transform the annotations made on the reference frame to be able to make
use of them on the video frame , 
params:
annotations - the premade annotations stored in the file, 
H - the transformation matrix between the reference frame and the given frame 
 '''
def transform_annotations(annotations, H):
    transformed_annotations = {}
    for key, polygons in annotations.items():
        transformed_annotations[key] = []
        for polygon in polygons:
            if not polygon:  
                continue
            pts = np.array(polygon, dtype=np.float32).reshape(-1, 1, 2)
            transformed_pts = cv2.perspectiveTransform(pts, H)
            transformed_annotations[key].append(transformed_pts.reshape(-1, 2).tolist())
    return transformed_annotations

### frame allignmnet

In [3]:
video__to_allign_1 = cv2.VideoCapture('../videos/easy_1.mp4')
video__to_allign_2 = cv2.VideoCapture('../videos/easy_2.mp4')
video__to_allign_3 = cv2.VideoCapture('../videos/easy_3.mp4')
video__to_allign_4 = cv2.VideoCapture('../videos/medium_1.mp4')
video__to_allign_5 = cv2.VideoCapture('../videos/medium_2.mp4')
video__to_allign_6 = cv2.VideoCapture('../videos/medium_3.mp4')
video__to_allign_7 = cv2.VideoCapture('../videos/hard_1.mp4')
video__to_allign_8 = cv2.VideoCapture('../videos/hard_2.mp4')
video__to_allign_9 = cv2.VideoCapture('../videos/hard_3.mp4')


'''function returning the given frame of the video usefull for computing the transformation matrix  between the video 
and the annotated reference pick'''
def give_frame(video):
    while True:
        ret_2, frame_to_allign = video.read()
        cv2.imshow('Frame', frame_to_allign)
        if frame_to_allign is None:
            break

        key = cv2.waitKey(30)
        if key == ord('q' ):
            break
    # cap.release()
    cv2.destroyAllWindows()
    return frame_to_allign

'''the allign frames  displayed'''
frame_to_allign_1 = give_frame(video__to_allign_1)
frame_to_allign_2 =  give_frame(video__to_allign_2)
frame_to_allign_3 = give_frame(video__to_allign_3)
frame_to_allign_4 = give_frame(video__to_allign_4)
frame_to_allign_5 = give_frame(video__to_allign_5)
frame_to_allign_6 = give_frame(video__to_allign_6)
frame_to_allign_7 = give_frame(video__to_allign_7)
frame_to_allign_8 = give_frame(video__to_allign_8)
frame_to_allign_9 = give_frame(video__to_allign_9)


# Annotation reading

here the reading of the annotation takes place, the annotation were made on the reference board image

In [4]:
import pickle

reference_pick = cv2.imread("../images/board_image.jpg")
reference_pick = cv2.resize(reference_pick, (512,512), interpolation = cv2.INTER_AREA)

with open('../resources/annotated_image.pkl', 'rb') as f:
    loaded_annotations = pickle.load(f)

with open('../resources/new_annon.pkl', 'rb') as f:
    new_loaded_annotations = pickle.load(f)

loaded_annotations = {key:val for key,val in loaded_annotations.items() if 'base' not in key}


'''so here with this weird code , we account for the the mistakes made when annotating, some typos etc
so auxillary fragmented annotations with corrections were made '''
new_loaded_annotations['red_start_1'] = new_loaded_annotations['']
new_loaded_annotations.pop('')
new_loaded_annotations

new_loaded_annotations = {
    key.replace('start', 'base'): value
    for key, value in new_loaded_annotations.items()
}

for key, value in new_loaded_annotations.items():
    loaded_annotations[key] = value




'''the content of the annotations showed'''
loaded_annotations


{'blue_start': [[[280, 594], [280, 554], [323, 557], [320, 594]]],
 'grid_1': [[[281, 553], [281, 516], [317, 514], [321, 555]]],
 'grid_2': [[[279, 516], [280, 476], [320, 476], [319, 516]]],
 'grid_3': [[[281, 476], [281, 438], [319, 436], [319, 476]]],
 'grid_4': [[[280, 476], [241, 474], [241, 437], [280, 437]]],
 'grid_5': [[[240, 475], [201, 475], [203, 436], [241, 436]]],
 'grid_6': [[[201, 436], [203, 398], [243, 399], [243, 434]]],
 'grid_7': [[[203, 399], [203, 358], [243, 359], [243, 400]]],
 'grid_8': [[[201, 398], [164, 398], [164, 360], [203, 358]]],
 'grid_9': [[[162, 397], [125, 398], [125, 360], [165, 360]]],
 'grid_10': [[[123, 398], [85, 398], [85, 359], [123, 360]]],
 'grid_11': [[[85, 359], [85, 321], [124, 319], [125, 358]]],
 'red_start': [[[85, 320], [83, 279], [127, 281], [123, 318]]],
 'grid_12': [[[123, 318], [162, 319], [164, 280], [125, 280]]],
 'grid_13': [[[162, 318], [202, 317], [203, 282], [163, 282]]],
 'grid_14': [[[201, 318], [242, 319], [242, 281], 

# The alligning frame and annotations

Here the annotation take place using introduced earlier auxillary function show_matches. The function basically relies on the sift algorithm used with euclidean norm and crosscheck() , the idea is to find the 200 best keypoints and based on them allign the frames. 200 seemed as big enough number to account for the possible inaccuracies caused by occlusions of grid , shadowing and light reflection 

In [5]:
''' function alligning the frame basing on the sift
params:
frame - frame to allign
reference_pick -  reference image  to allign the frame to '''
def allign_frame(frame, reference_pick,to_show=False):

    sift = cv2.SIFT_create()
    sift_kp1, sift_des1 = sift.detectAndCompute(reference_pick, None)
    sift_kp2, sift_des2 = sift.detectAndCompute(frame, None)
    bf_l2 = cv2.BFMatcher(cv2.NORM_L2,crossCheck=True)
    H, aligned_frame = show_matches(bf_l2, sift_kp1, sift_des1, sift_kp2, sift_des2,reference_pick,frame ,200)
    original_size = (1707*0.4, 1707*0.4)  
    resized_size = (512, 512)
    scaled_annotations = {}
    for key, polygons in loaded_annotations.items():
        scaled_annotations[key] = [
            [[int(pt[0] * resized_size[0] / original_size[0]), int(pt[1] * resized_size[1] / original_size[1])] for pt in polygon]
            for polygon in polygons
        ]

    transformed_annotations = transform_annotations(scaled_annotations, np.linalg.inv(H))
    aligned_frame_with_polygons = aligned_frame.copy()
    draw_polygons(aligned_frame_with_polygons, scaled_annotations)
    original_frame_with_polygons = frame.copy()
    draw_polygons(original_frame_with_polygons, transformed_annotations)

    return transformed_annotations

transformed_annotations_1 = allign_frame(frame_to_allign_1,reference_pick)
transformed_annotations_2 = allign_frame(frame_to_allign_2,reference_pick)
transformed_annotations_3 = allign_frame(frame_to_allign_3,reference_pick)
transformed_annotations_4 = allign_frame(frame_to_allign_4,reference_pick)
transformed_annotations_5 = allign_frame(frame_to_allign_5,reference_pick)
transformed_annotations_6 = allign_frame(frame_to_allign_6,reference_pick)
transformed_annotations_7 = allign_frame(frame_to_allign_7,reference_pick)
transformed_annotations_8 = allign_frame(frame_to_allign_8,reference_pick)
transformed_annotations_9 = allign_frame(frame_to_allign_9,reference_pick)


# Colorfinder

Here is stored the class  responsible for most of the work done concerning the detections. Basically the function filters
the image based on the indicated color ranges. It works in 3 colorspace, HSV,HLS,GBR and in two modes: The debbug mode with trackbar=True, allows for the manual changing of the color-ranges in the course of the video, thus seeing which elements are remaning in which color mask. It was basically used to find the proper ranges for all of the elements -pawns, dice, pawns in base. and the second mode with trackbar=False , is basically filterring the video for the provided colorrange, so it can be used when the desired color range is found in the debbug mode.

In [6]:
class ColorFinder:
    '''params:
    trackbar- boolean, when trackbar is set to true the debugg mode is present and the user can change the filter range of 
    the video, when false the video is run with predetermined values 
    colorspace-  the desired colorspace, since different elements can be filtered in different colorspaces, supports
    arguments : HSV, HLS, GBR
    red -boolean, so basially this argument is specifically designed to address the issue of the red color hue values
     in the hsv space. The issue is that the red color can either have very high hue or very low. So if we set red to True
      and set the min hue and max hue, the filtering for hue  instead of being [min,max] is the complement of this set so
      [0,min] + [max,max_possible_hue]'''


    def __init__(self, trackBar=False, colorSpace='HSV',red=False):
    
        self.trackBar = trackBar
        if red:
            self.red = True
        else:
            self.red = False
        self.colorSpace = colorSpace.upper()  
        if self.trackBar:
            self.initTrackbars()

    def empty(self, a):
        pass

    def initTrackbars(self):
        """Initialize the OpenCV trackbars for dynamic HSV/HLS/GBR value adjustment."""
        cv2.namedWindow("TrackBars")
        cv2.resizeWindow("TrackBars", 640, 240)
        

        if self.colorSpace == 'HLS':
            cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, self.empty)
            cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, self.empty)
            cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Light Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Light Max", "TrackBars", 255, 255, self.empty)
        elif self.colorSpace == 'GBR':
           
            cv2.createTrackbar("Green Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Green Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Blue Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Blue Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Red Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Red Max", "TrackBars", 255, 255, self.empty)
        else:
            cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, self.empty)
            cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, self.empty)
            cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Value Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Value Max", "TrackBars", 255, 255, self.empty)

    def getTrackbarValues(self):
        """Get the current color values set by the trackbars."""
       

        if self.colorSpace == 'HLS':
            hmin = cv2.getTrackbarPos("Hue Min", "TrackBars")
            smin = cv2.getTrackbarPos("Sat Min", "TrackBars")
            hmax = cv2.getTrackbarPos("Hue Max", "TrackBars")
            smax = cv2.getTrackbarPos("Sat Max", "TrackBars")
            lmin = cv2.getTrackbarPos("Light Min", "TrackBars")
            lmax = cv2.getTrackbarPos("Light Max", "TrackBars")
            colorVals = {"hmin": hmin, "smin": smin, "lmin": lmin,
                         "hmax": hmax, "smax": smax, "lmax": lmax}
        elif self.colorSpace == 'GBR':
            gmin = cv2.getTrackbarPos("Green Min", "TrackBars")
            gmax = cv2.getTrackbarPos("Green Max", "TrackBars")
            bmin = cv2.getTrackbarPos("Blue Min", "TrackBars")
            bmax = cv2.getTrackbarPos("Blue Max", "TrackBars")
            rmin = cv2.getTrackbarPos("Red Min", "TrackBars")
            rmax = cv2.getTrackbarPos("Red Max", "TrackBars")
            colorVals = {"gmin": gmin, "bmin": bmin, "rmin": rmin,
                         "gmax": gmax, "bmax": bmax, "rmax": rmax}
        else:
            hmin = cv2.getTrackbarPos("Hue Min", "TrackBars")
            smin = cv2.getTrackbarPos("Sat Min", "TrackBars")
            hmax = cv2.getTrackbarPos("Hue Max", "TrackBars")
            smax = cv2.getTrackbarPos("Sat Max", "TrackBars")
            vmin = cv2.getTrackbarPos("Value Min", "TrackBars")
            vmax = cv2.getTrackbarPos("Value Max", "TrackBars")
            colorVals = {"hmin": hmin, "smin": smin, "vmin": vmin,
                         "hmax": hmax, "smax": smax, "vmax": vmax}

        return colorVals

    def update(self, img, myColor=None):
        """Find a specified color in the given image."""
        imgColor = []
        mask = []

        if self.trackBar:
            myColor = self.getTrackbarValues()

        if myColor is not None:
            if self.colorSpace == 'HLS':
                imgColorSpace = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            elif self.colorSpace == 'GBR':
                imgColorSpace = img  
            else: 
                imgColorSpace = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            
            if self.colorSpace == 'GBR':
                lower = np.array([myColor['bmin'],myColor['gmin'],  myColor['rmin']])
                upper = np.array([myColor['bmax'],myColor['gmax'],  myColor['rmax']])
            elif self.colorSpace == 'HLS':
                lower = np.array([myColor['hmin'], myColor['lmin'], myColor['smin']])
                upper = np.array([myColor['hmax'], myColor['lmax'], myColor['smax']])
            else:  
                lower = np.array([myColor['hmin'], myColor['smin'], myColor['vmin']])
                upper = np.array([myColor['hmax'], myColor['smax'], myColor['vmax']])

            if self.red and self.colorSpace == 'HSV':
                lower_1 = np.zeros_like(lower)
                lower_2 = np.zeros_like(lower)
                upper_1 = np.zeros_like(lower)
                upper_2 = np.zeros_like(lower)
                upper_1[0] = lower[0] 
                upper_1[1:] = upper[1:]  
                lower_1[0] = 0           
                lower_1[1:] = lower[1:] 
                lower_2[0] = upper[0]    
                lower_2[1:] = lower[1:]  
                upper_2[0] = 179         
                upper_2[1:] = upper[1:] 
                mask_1 = cv2.inRange(imgColorSpace, lower_1, upper_1)
                mask_2 = cv2.inRange(imgColorSpace, lower_2, upper_2)
                imgColor_1 = cv2.bitwise_and(img, img, mask=mask_1)
                imgColor_2 = cv2.bitwise_and(img, img, mask=mask_2)
                imgColor = cv2.bitwise_or(imgColor_1,imgColor_2)
            else:
                mask = cv2.inRange(imgColorSpace, lower, upper)
                imgColor = cv2.bitwise_and(img, img, mask=mask)

        return imgColor, mask

## Colorfinder Test
Here the colorfinder is tested in the debugg mode so the functionality can be seen

In [7]:
import math
video_2 = cv2.VideoCapture('../videos/easy_1.mp4')

colorFinder = ColorFinder(trackBar=True, colorSpace='HSV')


while True:
    success, frame_3= video_2.read()
    if not success:
        print("Failed to capture frame. Exiting...")
        break

    


    
    gray_frame = cv2.cvtColor(frame_3, cv2.COLOR_BGR2GRAY)

    
 

    imgColor, mask = colorFinder.update(frame_3)
    imggray = cv2.cvtColor(imgColor,cv2.COLOR_BGR2GRAY)
    _,img_threshed = cv2.threshold(imggray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)) 

    img_threshed= cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)  
   
   
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_CLOSE, kernel)  
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel) 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)) 
    img_threshed = cv2.morphologyEx(img_threshed,cv2.MORPH_DILATE,kernel)
    

    cv2.imshow("Original Video", frame_3)
    cv2.imshow("Filtered Video", imgColor)
    cv2.imshow("thresholded", img_threshed)

        # Exit on pressing 'q'
    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# Detecting the elements of the gird
Here are the functions devouted to detecting the elements of the grid, since our logic was to filter out the respective elements by the color ranges they fall into, each element had to have separate category, these values here are already an effect of tedious work with the colorfinder in the debugg mode

In [8]:
def detect_blue_base(frame, annotations, frame_to_annotate):
    bounding_boxes_blue = []
    vals_blue_start = {'rmin': 0, 'rmax': 6, "bmin": 45, "bmax": 255, 'gmax': 185, 'gmin': 0}
    colorFinder_blue_start = ColorFinder(trackBar=False, colorSpace='GBR')
    frame_gauss = cv2.GaussianBlur(frame, (15, 15), 0)
    
    # filtering the area of interest since we only search for the pawns in the blue base and blue start in this case 
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    annotations_for_blue_start = {key: value for key, value in annotations.items() if 'blue_base' in key or 'blue_start' in key }
    for _, points in annotations_for_blue_start.items():
        points_array = np.array(points, dtype=np.int32)
        cv2.fillPoly(mask, [points_array], 255)

    masked_frame = cv2.bitwise_and(frame_gauss, frame_gauss, mask=mask)
    imgColor, color_mask = colorFinder_blue_start.update(masked_frame,vals_blue_start)
    imggray = cv2.cvtColor(imgColor, cv2.COLOR_BGR2GRAY)
    _, img_threshed = cv2.threshold(imggray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    #Morphological operations 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_CLOSE, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_DILATE, kernel)
    contours, _ = cv2.findContours(img_threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #finding contours 
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes_blue.append((x,y,w,h))
        #drawing the bounding boxes for detected contours 
        cv2.rectangle(frame_to_annotate, (x, y), (x + w, y + h), (255, 0, 0), 2)
    return frame_to_annotate,bounding_boxes_blue

In [39]:
def detect_red_base(frame,annotations,frame_to_annotate):
    boundig_boxes_red = []
    vals_red_start = {'hmin': 53, 'hmax': 132, 'smin': 220, 'smax': 255, 'vmin': 0, 'vmax': 118}
    vals_red_start = {'hmin': 53, 'hmax': 132, 'smin': 220, 'smax': 255, 'vmin': 0, 'vmax': 108}
    colorFinder_red_start = ColorFinder(trackBar=False, colorSpace='HSV', red=True)
    frame_3_gauss = cv2.GaussianBlur(frame, (11, 11), 0)

    #filtering the area of intereset since we search only in red base and red start 
    mask = np.zeros(frame_3.shape[:2], dtype=np.uint8)
    annotations_for_red_start= {key: value for (key, value) in annotations.items() if  'red_start'in key  or 'red_base' in key}
    for _, points in annotations_for_red_start.items():
        points_array = np.array(points, dtype=np.int32)
        cv2.fillPoly(mask, [points_array], 255)
    masked_frame = cv2.bitwise_and(frame_3_gauss, frame_3_gauss, mask=mask)
    imgColor, color_mask = colorFinder_red_start.update(masked_frame, vals_red_start)
    imggray = cv2.cvtColor(imgColor, cv2.COLOR_BGR2GRAY)
    _, img_threshed = cv2.threshold(imggray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)  
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_CLOSE, kernel)  
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_DILATE, kernel)

    #Finding contours 
    contours, _ = cv2.findContours(img_threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 50:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        boundig_boxes_red.append((x,y,w,h))
        # Draw bounding boxes for detected contours 
        cv2.rectangle(frame_to_annotate, (x, y), (x + w, y + h), (0, 0, 255), 2)

    return frame_to_annotate,boundig_boxes_red
    

In [10]:
def detect_blue_pawns(frame,annotations,frame_annotated):
    mask_blue = np.zeros(frame.shape[:2], dtype=np.uint8)
    current_annotations_blue = {key: val for key, val in annotations.items() if 'grid' in key}
    # Filtering the are of interest since we only search for the pawns on the grid 
    for polygon_name_blue, points_blue in current_annotations_blue.items():
        points_array_blue = np.array(points_blue, dtype=np.int32)
        cv2.fillPoly(mask_blue, [points_array_blue], 255)
    colorFinder_blue = ColorFinder(trackBar=False, colorSpace='HSV')
    vals_blue = {'hmin': 95, 'hmax': 112, 'smin': 140, 'smax': 255, 'vmin': 0, 'vmax': 156}
    masked_frame_blue = cv2.bitwise_and(frame, frame, mask=mask_blue)
    imgColor_blue, mask_blue = colorFinder_blue.update(masked_frame_blue, vals_blue)
    imggray_blue = cv2.cvtColor(imgColor_blue, cv2.COLOR_BGR2GRAY)
    _, img_threshed_blue = cv2.threshold(imggray_blue, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations 
    kernel_blue = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_threshed_blue = cv2.morphologyEx(img_threshed_blue, cv2.MORPH_OPEN, kernel_blue)
    kernel_blue = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    img_threshed_blue = cv2.morphologyEx(img_threshed_blue, cv2.MORPH_CLOSE, kernel_blue)
    img_threshed_blue = cv2.morphologyEx(img_threshed_blue, cv2.MORPH_OPEN, kernel_blue)
    kernel_blue = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    img_threshed_blue = cv2.morphologyEx(img_threshed_blue, cv2.MORPH_DILATE, kernel_blue)

    # Contour detection 
    contours_blue, _ = cv2.findContours(img_threshed_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes_blue = []
    for cnt_blue in contours_blue:
        area_blue = cv2.contourArea(cnt_blue)
        if 50 < area_blue < 400:  
            perimeter_blue = cv2.arcLength(cnt_blue, True)
            if perimeter_blue == 0:
                continue
            circularity_blue = 4 * np.pi * (area_blue / (perimeter_blue ** 2))
            if 0.4 < circularity_blue < 0.9:  
                x_blue, y_blue, w_blue, h_blue = cv2.boundingRect(cnt_blue)
                bounding_boxes_blue.append((x_blue, y_blue, w_blue, h_blue))
                #drawing the bounding boxes for the deteced contours 
                cv2.rectangle(frame_annotated, (x_blue, y_blue), (x_blue + w_blue, y_blue + h_blue), (255, 0, 0), 2)
    return frame_annotated,bounding_boxes_blue
    

In [37]:
def detect_red_pawns(frame,annotations,frame_annotated):
    vals_red  = {'hmin': 9, 'hmax': 165, 'smin': 180, 'smax': 255, 'vmin': 80, 'vmax': 140}
    vals_red  = {'hmin': 9, 'hmax': 165, 'smin': 200, 'smax': 255, 'vmin': 80, 'vmax': 140}
    colorFinder_red = ColorFinder(trackBar=False, colorSpace='HSV',red='True')
    
    #filtering the area of interest since we search for the red pawns only in the grid tiles 
    mask_red = np.zeros(frame.shape[:2], dtype=np.uint8)
    current_annotations_red = {key: val for key, val in annotations.items() if 'red' not in key and 'base' not in key }
    for polygon_name_red, points_red in current_annotations_red.items():
        points_array_red = np.array(points_red, dtype=np.int32)
        cv2.fillPoly(mask_red, [points_array_red], 255)
    masked_frame_red = cv2.bitwise_and(frame, frame, mask=mask_red)
    imgColor_red, mask_red = colorFinder_red.update(masked_frame_red, vals_red)
    imggray_red = cv2.cvtColor(imgColor_red, cv2.COLOR_BGR2GRAY)
    _, img_threshed_red = cv2.threshold(imggray_red, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations 
    kernel_red = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img_threshed_red = cv2.morphologyEx(img_threshed_red, cv2.MORPH_CLOSE, kernel_red)  # Close gaps
    img_threshed_red = cv2.morphologyEx(img_threshed_red, cv2.MORPH_OPEN, kernel_red)
    kernel_red = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6))
    img_threshed_red = cv2.morphologyEx(img_threshed_red, cv2.MORPH_DILATE, kernel_red)

    # Contour detection 
    contours_red, _ = cv2.findContours(img_threshed_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes_red = []

    for cnt_red in contours_red:
        area_red = cv2.contourArea(cnt_red)
        if 100 < area_red < 500:  
            perimeter_red = cv2.arcLength(cnt_red, True)
            if perimeter_red == 0:
                continue
            circularity_red = 4 * np.pi * (area_red / (perimeter_red ** 2))
            if 0.5 < circularity_red < 0.9:  
                x_red, y_red, w_red, h_red = cv2.boundingRect(cnt_red)
                bounding_boxes_red.append((x_red, y_red, w_red, h_red))
                #drawing the bounding boxes for the detected contours 
                cv2.rectangle(frame_annotated, (x_red, y_red), (x_red + w_red, y_red + h_red), (0, 0, 255), 2)
    return frame_annotated,bounding_boxes_red



In [12]:

def detect_pips(dice_roi, circularity_threshold=0.5):
  
    gray_roi = cv2.cvtColor(dice_roi, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_roi, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
    binary_mask = cv2.morphologyEx(binary_mask,cv2.MORPH_OPEN,kernel)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_mask = np.zeros_like(binary_mask)
    pip_count = 0
    
    #validation of contours 
    for i,contour in enumerate(contours):
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, closed=True)
        if perimeter > 0:  
            circularity = (4 * np.pi * area) / (perimeter ** 2)
        else:
            circularity = 0
        #sophisticated hand-crafted conditions when to assess the contour as a valid dice pip candidate
        if area > 10 or area <1 or (circularity < 0.6 and area <= 2.5):
            continue
        if circularity >= circularity_threshold:
            pip_count += 1
            # Draw the contour on the mask for visualization
            cv2.drawContours(contour_mask, [contour], -1, 255, thickness=cv2.FILLED)
    if pip_count ==0:
        return None, binary_mask
    return min(pip_count,6), binary_mask









In [13]:

''''only the dice detected'''
def detect_dice(frame, frame_annotated, mask_binary_white, tracker, tracking, frame_counter):
    colorFinder = ColorFinder(trackBar=False, colorSpace='HLS')
    vals = {'hmin': 107, 'hmax': 140, 'smin': 12, 'smax': 255, 'lmin': 140, 'lmax': 255}
    fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=3, detectShadows=True)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    reset_interval =9  # Number of frames after which the tracker is reinitialized

    # Background subtraction
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    bg_subtracted_frame = cv2.bitwise_and(frame, frame, mask=fgmask)

    frame_to_track = frame.copy()
    # Color filtering on the original frame
    imgColor, mask = colorFinder.update(bg_subtracted_frame, vals)
    frame_to_track, _ = colorFinder.update(frame_to_track, vals)
    imggray = cv2.cvtColor(imgColor, cv2.COLOR_BGR2GRAY)
    _, img_threshed = cv2.threshold(imggray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_CLOSE, kernel)
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_DILATE, kernel)
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_DILATE, kernel)

   
    combined_mask = cv2.bitwise_and(img_threshed, fgmask)
    combined_mask = cv2.bitwise_and(combined_mask, cv2.bitwise_not(mask_binary_white))
    if tracking and frame_counter % reset_interval != 0:
        # if tracker succesfull and not time to reset tracker than bounding box for te detected dice
        success, bbox = tracker.update(frame_to_track)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            # Draw tracking bounding box
            cv2.rectangle(frame_annotated, (x - 5, y - 5), (x + w + 5, y + h + 5), (255, 255, 255), 2)  # Blue for tracking
        else:
            # If object lost trakcing false
            tracking = False
            tracker = None

    if not tracking or frame_counter % reset_interval == 0:
        # Detecting the contoursagain since no tracking or the reset interval has happened 
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the largest contour, as we can have only once dice at time right?
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 30:
                x, y, w, h = cv2.boundingRect(largest_contour)
                # KFC Tracker initialized  on the normal color frame since , if object found we do not want it
                # to disappear with the background subtraction
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame_to_track, (x, y, w, h))  
                tracking = True
                frame_counter = 0  
                cv2.rectangle(frame_annotated, (x - 5, y - 5), (x + w + 5, y + h + 5), (255, 255, 255), 2)  # Green for new tracking box
        else:
            tracking = False 
    return frame_annotated, tracker, tracking




'''the same logic as in the function above with the change that also the pips are detected
in case the tracker is stable on an object and remain in one place, the threshold movement is small '''
def detect_dice_with_pips(frame, frame_annotated, mask_binary_white, tracker, tracking, frame_counter, stable_count, dice_pips, prev_bbox, calculated):
    colorFinder = ColorFinder(False, 'HLS')
    vals = {'hmin': 107, 'hmax': 140, 'smin': 12, 'smax': 255, 'lmin': 140, 'lmax': 255}
    dice_image = np.zeros_like(frame)
    frame_to_count_pips = frame.copy()

    fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=3, detectShadows=True)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    reset_interval = 7 # Number of frames after which the tracker is reinitialized

    # Background subtraction and preprocessing
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    bg_subtracted_frame = cv2.bitwise_and(frame, frame, mask=fgmask)

    # Color filtering on the original frame
    imgColor, mask = colorFinder.update(bg_subtracted_frame, vals)
    #imgColor, mask = colorFinder.update(frame, vals)

    imggray = cv2.cvtColor(imgColor, cv2.COLOR_BGR2GRAY)
    _, img_threshed = cv2.threshold(imggray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_CLOSE, kernel)

    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_OPEN, kernel)
    img_threshed = cv2.morphologyEx(img_threshed, cv2.MORPH_DILATE, kernel)
    
    combined_mask = cv2.bitwise_and(img_threshed, cv2.bitwise_not(mask_binary_white))


    if tracking and frame_counter % reset_interval != 0:
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            # Draw tracking box
            cv2.rectangle(frame_annotated, (x-5, y-5), (x + w+5, y + h+5), (255, 255, 255), 2)

            # Check if the bounding box is stable for at least 5frames
            stable_count += 1
            if stable_count >=5 :  
                if prev_bbox and abs(prev_bbox[0] - x) < 10 and  abs(prev_bbox[1] - y) < 10:
                    if not calculated:
                        if y> 5 and y+h+5< frame.shape[0] and x >5 and x+w+5< frame.shape[1]:
                            frame_to_count_pips = frame[y-5:y+h+5, x-5:x+w+5]
                            dice_pips, dice_image = detect_pips(frame_to_count_pips)  # Call pip detection function
                            if dice_pips is not None:
                                cv2.putText(frame_annotated, f"Pips: {dice_pips}", (x, y - 10),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                                calculated = True  

                # Update prev_bbox if bounding box is stable
            prev_bbox = (x, y, w, h)
        else:
            tracking = False
            tracker = None
            stable_count = 0
            calculated = False  

    if not tracking or frame_counter % reset_interval == 0:
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 30:
                x, y, w, h = cv2.boundingRect(largest_contour)
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, (x, y, w, h))
                tracking = True
                stable_count = 0
            if not frame_counter % reset_interval == 0:
                calculated = False  # Reset flag on tracker initialization

    # Proximity check: Reset flag if the bounding box has moved significantly
    if prev_bbox is not None:
        x, y, w, h = prev_bbox
        if abs(prev_bbox[0] - x) > 50 or abs(prev_bbox[1] - y) > 50:
            calculated = False  

    return frame_annotated, tracker, tracking, stable_count, dice_pips, dice_image, prev_bbox, calculated


    

In [14]:
'''the logic identicall to detectin red blue and green only the parameters differ'''
def detect_yellow_pawns(frame,annotations,frame_annotated):
    bounding_boxes_yellow = []
    mask_yellow = np.zeros(frame.shape[:2], dtype=np.uint8)
    current_annotations_yellow = {key: val for key, val in annotations.items() if 'yellow'  in key or 'grid' in key or 'start' in key}
    colorFinder_yellow = ColorFinder(trackBar=False, colorSpace='HSV')
    vals_yellow = {'hmin': 14, 'hmax': 24, 'smin': 190, 'smax': 255, 'vmin': 0, 'vmax': 176}
    #filtering the area of interest 
    for polygon_name_yellow, points_yellow in current_annotations_yellow.items():
        points_array_yellow = np.array(points_yellow, dtype=np.int32)
        cv2.fillPoly(mask_yellow, [points_array_yellow], 255)
    masked_frame_yellow = cv2.bitwise_and(frame, frame, mask=mask_yellow)

    #
    imgColor_yellow, _ = colorFinder_yellow.update(masked_frame_yellow, vals_yellow)
    imggray_yellow = cv2.cvtColor(imgColor_yellow, cv2.COLOR_BGR2GRAY)
    _, img_threshed_yellow = cv2.threshold(imggray_yellow, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    img_threshed_yellow = cv2.morphologyEx(img_threshed_yellow, cv2.MORPH_OPEN, kernel)  
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    img_threshed_yellow = cv2.morphologyEx(img_threshed_yellow, cv2.MORPH_CLOSE, kernel)  
    img_threshed_yellow = cv2.morphologyEx(img_threshed_yellow, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img_threshed_yellow = cv2.morphologyEx(img_threshed_yellow, cv2.MORPH_DILATE, kernel)

    # Contour detection 

    contours_yellow, _ = cv2.findContours(img_threshed_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours_yellow = sorted(contours_yellow, key=cv2.contourArea, reverse=True)[:4]
    for cnt in contours_yellow:
        area_yellow = cv2.contourArea(cnt)
        if area_yellow > 75: 
            perimeter_yellow = cv2.arcLength(cnt, True)
            if perimeter_yellow == 0:  
                continue
            circularity_yellow = 4 * np.pi * (area_yellow / (perimeter_yellow ** 2))

            if circularity_yellow > 0.2:  
                x_yellow, y_yellow, w_yellow, h_yellow = cv2.boundingRect(cnt)
                bounding_boxes_yellow.append((x_yellow, y_yellow, w_yellow, h_yellow))
                cv2.rectangle(frame_annotated, (x_yellow, y_yellow), (x_yellow + w_yellow, y_yellow + h_yellow), (0, 255, 255), 2)
    return frame_annotated, bounding_boxes_yellow
    pass

In [15]:
'''the same as detectin red ,blue,yellow but  with different params'''
def detect_green_pawns(frame,annotations,frame_annotated):
    vals_green = {'hmin':38,'hmax':90,'smin':131,'smax':255,'vmin':67,'vmax':115}
    colorFinder_green = ColorFinder(trackBar=False, colorSpace='HSV')  

    current_annotations_green = {key: val for key, val in annotations.items() if 'green'  in key or 'grid' in key or 'start' in key}
    mask_green = np.zeros(frame.shape[:2], dtype=np.uint8)

   
    for polygon_name_green, points_green in current_annotations_green.items():
        points_array_green = np.array(points_green, dtype=np.int32)
        cv2.fillPoly(mask_green, [points_array_green], 255)
    
    masked_frame_green = cv2.bitwise_and(frame, frame, mask=mask_green)
    masked_frame_green = cv2.bitwise_and(frame, frame, mask=mask_green)
    imgColor_green, mask_green = colorFinder_green.update(masked_frame_green, vals_green)
    imggray_green = cv2.cvtColor(imgColor_green, cv2.COLOR_BGR2GRAY)
    _, img_threshed_green = cv2.threshold(imggray_green, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations for green
    kernel_green = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    img_threshed_green = cv2.morphologyEx(img_threshed_green, cv2.MORPH_OPEN, kernel_green)
    kernel_green = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    img_threshed_green = cv2.morphologyEx(img_threshed_green, cv2.MORPH_CLOSE, kernel_green)
    img_threshed_green = cv2.morphologyEx(img_threshed_green, cv2.MORPH_OPEN, kernel_green)
    img_threshed_green = cv2.morphologyEx(img_threshed_green, cv2.MORPH_DILATE, kernel_green)

    # Contour detection for green
    contours_green, _ = cv2.findContours(img_threshed_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_green = sorted(contours_green, key=cv2.contourArea, reverse=True)[:4]
    bounding_boxes_green = []
    for cnt_green in contours_green:
        area_green = cv2.contourArea(cnt_green)
        if area_green > 50 : 
            perimeter_green = cv2.arcLength(cnt_green, True)
            if perimeter_green == 0:
                continue
            circularity_green = 4 * np.pi * (area_green / (perimeter_green ** 2))
            if circularity_green > 0.1:
                x_green, y_green, w_green, h_green = cv2.boundingRect(cnt_green)
                bounding_boxes_green.append((x_green, y_green, w_green, h_green))
                cv2.rectangle(frame_annotated, (x_green, y_green), (x_green + w_green, y_green + h_green), (0, 255, 0), 2)

    return frame_annotated,bounding_boxes_green

# The detection of pawns and dice only

here the experimenting with detecting the dice and the pawns take place 

In [16]:
video_2 = cv2.VideoCapture('../videos/medium_3_Trim.mp4')
current_annotations = transformed_annotations_6
colorFinder = ColorFinder(False,'HLS')
success, first_frame = video_2.read()
if not success:
    print("Failed to capture first frame. Exiting...")
    exit()

# Apply ColorFinder to get the mask based on the first frame
vals_2 = {'hmin': 107, 'hmax': 140, 'smin': 0, 'smax': 255, 'lmin': 140, 'lmax': 255}
imgColor, mask = colorFinder.update(first_frame, vals_2)
mask_binary_white = cv2.inRange(mask, 1, 255)  
mask_binary_white = cv2.morphologyEx(mask_binary_white, cv2.MORPH_DILATE, kernel, iterations=3)
cv2.imshow("white",mask_binary_white)
cv2.waitKey(1)


frame_counter = 1
tracker = None
tracking = False
while True:
    success_blue, frame_3 = video_2.read()
    if not success_blue:
        print("Failed to capture frame. Exiting...")
        break
    
    frame_annotated  = frame_3.copy()
    frame_annotated,_ = detect_red_base(frame_3,current_annotations,frame_annotated)
    frame_annotated,_ = detect_blue_base(frame_3,current_annotations,frame_annotated)
    frame_annotated,_ = detect_blue_pawns(frame_3,current_annotations,frame_annotated)
    frame_annotated,_ = detect_red_pawns(frame_3,current_annotations,frame_annotated)
    frame_annotated,_ = detect_green_pawns(frame_3,current_annotations,frame_annotated)
    frame_annotated,_ = detect_yellow_pawns(frame_3,current_annotations,frame_annotated)
    frame_annotated,tracker,tracking = detect_dice(frame_3,frame_annotated,mask_binary_white,tracker,tracking,frame_counter)
    frame_counter+=1
    
    if frame_counter %100 ==0:
        current_annotations = allign_frame(frame_3,reference_pick)
    cv2.imshow("annotated frame",frame_annotated)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# All the detections  without the state game control

In [17]:
video_2 = cv2.VideoCapture('../videos/easy_1_Trim.mp4')
current_annotations = transformed_annotations_1
colorFinder = ColorFinder(False, 'HLS')

success, first_frame = video_2.read()
if not success:
    print("Failed to capture first frame. Exiting...")
    exit()

# Apply ColorFinder to get the mask based on the first frame
vals_2 = {'hmin': 107, 'hmax': 140, 'smin': 0, 'smax': 255, 'lmin': 140, 'lmax': 255}
imgColor, mask = colorFinder.update(first_frame, vals_2)
mask_binary_white = cv2.inRange(mask, 1, 255)  # Convert to binary mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
mask_binary_white = cv2.morphologyEx(mask_binary_white, cv2.MORPH_DILATE, kernel, iterations=4)
cv2.imshow("white", mask_binary_white)
cv2.waitKey(1)

# Initialize variables for dice detection and tracking
frame_counter = 1
tracker = None
tracking = False
stable_count = 3
dice_pips = None
prev_bbox = None  # Initialize prev_bbox to track bounding box changes
calculated = False  # Flag to track if pips have already been calculated

# Main processing loop
while True:
    success_blue, frame_3 = video_2.read()
    if not success_blue:
        print("Failed to capture frame. Exiting...")
        break

    frame_annotated = frame_3.copy()

    # if frame_counter > 1200:
        # Perform other detections
    frame_annotated,_ = detect_red_base(frame_3, current_annotations, frame_annotated)
    frame_annotated,_ = detect_blue_base(frame_3, current_annotations, frame_annotated)
    frame_annotated,_ = detect_blue_pawns(frame_3, current_annotations, frame_annotated)
    frame_annotated,_ = detect_red_pawns(frame_3, current_annotations, frame_annotated)
    frame_annotated,_ = detect_green_pawns(frame_3, current_annotations, frame_annotated)
    frame_annotated,_ = detect_yellow_pawns(frame_3, current_annotations, frame_annotated)

    # Perform dice detection with pip counting
    frame_annotated, tracker, tracking, stable_count, dice_pips, dice_img, prev_bbox, calculated = detect_dice_with_pips(
        frame_3, frame_annotated, mask_binary_white, tracker, tracking, frame_counter, stable_count, dice_pips, prev_bbox, calculated
    )
    cv2.imshow("dice img",dice_img)
    if dice_pips is not None:
            print(f'number of pips : {dice_pips}')
            dice_pips = None

    # Update annotations periodically
    if frame_counter % 100 == 0:
        current_annotations = allign_frame(frame_3, reference_pick)

    # Display the annotated frame
    cv2.imshow("annotated frame", frame_annotated)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    frame_counter += 1

cv2.destroyAllWindows()


# State of the game control

Here all of the previous elements are gethered and the whole logic takes place.


In [18]:

'''the function responsible for decidin whether update the state on the game given the detections, and state of the game'''
def update_grid_states_simple(detections, grid_states, annotations, overlap_threshold=0.1, area_threshold=0.3):
    
    updated_states = grid_states
    grid_polygons = {key: Polygon(coords) for key, coords in annotations.items()}
    for x, y, w, h in detections:
        bbox = box(x, y, x + w, y + h)
        best_grid_id = None
        max_overlap = 0

        for grid_id, polygon in grid_polygons.items():
            intersection_area = bbox.intersection(polygon).area
            bbox_area = bbox.area
            overlap_ratio = intersection_area / bbox_area
            tile_area = polygon.area
            intersection_to_tile_ratio = intersection_area / tile_area

            if overlap_ratio > max_overlap:
                max_overlap = overlap_ratio
                best_grid_id = grid_id

            if intersection_to_tile_ratio >= area_threshold and 'base' in grid_id:
                updated_states[grid_id] = True

        if best_grid_id and max_overlap >= overlap_threshold:
            updated_states[best_grid_id] = True

    return updated_states



In [19]:
'''function used for annotating the state of the game frame, basically the centers of the respective grid tiles are found 
for annotation'''
def get_centroid(coords):
    coords = np.array(coords)
    x = coords[:, 0]
    y = coords[:, 1]
    centroid_x = int(np.mean(x))
    centroid_y = int(np.mean(y))
    return (centroid_x, centroid_y)

'''the function to reset the state of detections after each frame'''
def reset_detections(dictionary):
    reseted_dictionary = {key:False for key in dictionary}
    return reseted_dictionary


In [20]:
'''getting the scaled annotations to be able to insert them on the refernce pick'''
original_size = (1707*0.4, 1707*0.4)  
resized_size = (512, 512)
scaled_annotations = {}
for key, polygons in loaded_annotations.items():
    scaled_annotations[key] = [
        [[int(pt[0] * resized_size[0] / original_size[0]), int(pt[1] * resized_size[1] / original_size[1])] for pt in polygon]
        for polygon in polygons
        ]


In [21]:
'''the history of the states of the grid tiles is updated, so basically there is some thershold of history for
each grid tile stored, it stores True,False refering to either the tile was occupied or not
the updating adds the most recent event to the history and in case the history is longer than threshold it erases the
least recent event from the story'''
def update_history(grid_history,grid_state):
    
    if len(list(grid_history.values())[0]) >=120:
        for key,val in grid_state.items():
            grid_history[key].append(val)
            grid_history[key].pop(0)
            
    else:
        for key,val in grid_state.items():
            grid_history[key].append(val)
            
    return grid_history

In [22]:
'''updating the changes , so the history of the changes of the grid states is also stored, and it helps determinging
whether pawn moved or not '''
def update_changes(grid_state,grid_history,grid_changes,stability_threshold=60,change_history=80,negative_stability=50):
    # 40,50,30

    key_to_change = None
    for  key,val in  grid_state.items():
        if val and not any(grid_history[key][-negative_stability:]):
            grid_changes[key] =True
        elif grid_changes[key]:
            recent_history = grid_history[key][-change_history:]
            if sum(recent_history) >= stability_threshold:
                key_to_change = key 
                grid_changes[key] = False  
                break
    return grid_changes,key_to_change
            

In [23]:
'''this function ensures that there are 4 pawns of the given color at given instance on the gird,
it works in the following way. In case that less than 4 pawns are detected then we look in the history and
find the most recent tile that has been occupied by the given color pawn but is not occupied now, and we infer the pawns
is most likely there, only the contour has vanished due to shadowing occlucion etc.
In case more than 4 pawns are detected than we do in different way, look at the detected tiles, and we want
to dispose of the false  detections in bases, because they tend to be oversensitive - it is difficult to detect blue pawns
of nearly the same hue as the tile it lays on and causes true positives. So we see the tiles which are now true but the 
last time they were true was longest ago'''
def ensure_minimum_keys_true(grid_states, grid_history, min_true=4):
    true_keys = [key for key, value in grid_states.items() if value]
    num_true = len(true_keys)
    

    if num_true == min_true:
        return grid_states
    elif num_true > min_true:
        keys_needed = num_true - min_true
        candidate_keys = [
        key for key, history in grid_history.items()
        if  grid_states[key] and 'base' in key and not any(history) 
        ]
        if candidate_keys is not None:
            keys_needed = max(keys_needed - len(candidate_keys),0)
            for key in candidate_keys[:keys_needed]:
                grid_states[key] = False
        if  keys_needed ==0:
            return grid_states
        candidate_keys = [
        key for key, history in grid_history.items()
        if  grid_states[key] and 'base' in key and any(history) 
        ]
        candidate_keys.sort(key=lambda key: max(idx for idx, val in enumerate(grid_history[key]) if val))
        for key in candidate_keys[:keys_needed]:
            grid_states[key] = False
        return grid_states

    

    else:
        keys_needed = min_true - num_true
        candidate_keys = [
            key for key, history in grid_history.items()
            if not grid_states[key] and any(history) 
        ]
        
        candidate_keys.sort(key=lambda key: max(idx for idx, val in enumerate(grid_history[key]) if val), reverse=True)
        for key in candidate_keys[:keys_needed]:
            grid_states[key] = True
        
        return grid_states


# The tracking and  game state detection system

In [69]:




'''here different annotations must be choosen for the different videos
the annotations are not hardcoded but are calculated with the sift in the previous cells with respect to the
annotated reference pick , '''
video_2 = cv2.VideoCapture('../videos/hard_3.mp4')



current_annotations = transformed_annotations_9
colorFinder = ColorFinder(False, 'HLS')

success, first_frame = video_2.read()
if not success:
    print("Failed to capture first frame. Exiting...")
    exit()


'''the white mask extraction from the first frame , to surpress the possibility of the false
positive dice detections , as the filtering of the white dice is not possible with the white
tape knife with the nearly the same parameters in each of the colorspaces: HSV, HLS, GBR'''
vals_2 = {'hmin': 107, 'hmax': 140, 'smin': 0, 'smax': 255, 'lmin': 140, 'lmax': 255}
imgColor, mask = colorFinder.update(first_frame, vals_2)
mask_binary_white = cv2.inRange(mask, 1, 255)  
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
mask_binary_white = cv2.morphologyEx(mask_binary_white, cv2.MORPH_DILATE, kernel, iterations=4)


'''initialization of the all variables need to track the state of the game '''
frame_counter = 1
tracker = None
tracking = False
stable_count = 3
dice_pips = None
prev_bbox = None  #
calculated = False  
history_limit = 180
last_dice_pips = 'not yet'
pawn_movement = 'not yet moved'
red_moved_to = None
blue_moved_to = None
green_moved_to = None
yellow_moved_to = None
movement_color = (255, 255, 255)
entered_the_game = None


'''initializing the historeis '''
grid_histories_red = dict((k, [False]) if 'red_base' not in k  else (k, [True]) for k in current_annotations.keys())
grid_histories_blue = dict((k, [False]) if 'blue_base' not in k  else (k, [True])  for k in current_annotations.keys())
grid_histories_green = dict((k, [False]) if 'green_base' not in k  else (k, [True]) for k in current_annotations.keys())
grid_histories_yellow = dict((k, [False]) if 'yellow_base' not in k  else (k, [True]) for k in current_annotations.keys())


'''initializing the states '''
grid_states_red = {key:False for key in current_annotations.keys()}
grid_states_blue = {key: False for key in current_annotations.keys()}
grid_states_green = {key: False for key in current_annotations.keys()}
grid_states_yellow = {key: False for key in current_annotations.keys()}

'''initializing the changes for the message of the pawns movement'''
grid_changes_red = {key:False for key in current_annotations.keys()}
grid_changes_blue = {key: False for key in current_annotations.keys()}
grid_changes_green = {key: False for key in current_annotations.keys()}
grid_changes_yellow = {key: False for key in current_annotations.keys()}

'''initialized the ensured histories, which are the historeis but accounting for the correction from the 
 ensure minimum keys ,so alwasy 4 detectons for each color '''

grid_histories_red_ensured = dict((k, [False]) if 'red_base' not in k  else (k, [True]) for k in current_annotations.keys())
grid_histories_blue_ensured = dict((k, [False]) if 'blue_base' not in k  else (k, [True])  for k in current_annotations.keys())
grid_histories_green_ensured = dict((k, [False]) if 'green_base' not in k  else (k, [True]) for k in current_annotations.keys())
grid_histories_yellow_ensured = dict((k, [False]) if 'yellow_base' not in k  else (k, [True]) for k in current_annotations.keys())



'''changing the format of the annotations to be valid for the shapely polygons creation'''
grid_polygons = {}
for key, coords in current_annotations.items():
    try:
        grid_polygons[key] = current_annotations[key][0]
    except Exception as e:
        print(f"Error creating polygon for {key} with coordinates {coords}: {e}")



'''the loop woth all the logic '''
while True:
    game_control_frame = reference_pick.copy()
    success_blue, frame_3 = video_2.read()
    if not success_blue:
        print("Failed to capture frame. Exiting...")
        break

    frame_annotated = frame_3.copy()
    frame_tracked_state = frame_3.copy()

    '''the detection only starts after some predefined times since firstly the pawns must be set on the grid '''
    if frame_counter > 1: #1200
        '''detect pawns of all the colors'''
        frame_annotated, red_detections = detect_red_pawns(frame_3, current_annotations, frame_annotated)
        frame_annotated, blue_detections = detect_blue_pawns(frame_3, current_annotations, frame_annotated)
        frame_annotated, green_detections = detect_green_pawns(frame_3, current_annotations, frame_annotated)
        frame_annotated, yellow_detections = detect_yellow_pawns(frame_3, current_annotations, frame_annotated)
        frame_annotated,red_base_detections = detect_red_base(frame_3, current_annotations, frame_annotated)
        frame_annotated,blue_base_detections= detect_blue_base(frame_3, current_annotations, frame_annotated)

        '''Update grid states for each color'''
        if red_base_detections:
             grid_states_red = update_grid_states_simple(red_base_detections, grid_states_red, grid_polygons)
        if blue_base_detections:
             grid_states_blue = update_grid_states_simple(blue_base_detections, grid_states_blue, grid_polygons)
        if red_detections:
             grid_states_red = update_grid_states_simple(red_detections, grid_states_red, grid_polygons)
        if blue_detections:
            grid_states_blue = update_grid_states_simple(blue_detections, grid_states_blue, grid_polygons)
        if yellow_detections:
            grid_states_yellow = update_grid_states_simple(yellow_detections, grid_states_yellow, grid_polygons)
        if green_detections:
            grid_states_green = update_grid_states_simple(green_detections, grid_states_green, grid_polygons)

        
        '''updating the grid changes for each color '''
        grid_changes_red,key_to_show_red = update_changes(grid_states_red,grid_histories_red_ensured,grid_changes_red)
        grid_changes_blue,key_to_show_blue = update_changes(grid_states_blue,grid_histories_blue_ensured,grid_changes_blue)
        grid_changes_green,key_to_show_green = update_changes(grid_states_green,grid_histories_green_ensured,grid_changes_green)
        grid_changes_yellow,key_to_show_yellow = update_changes(grid_states_yellow,grid_histories_yellow_ensured,grid_changes_yellow)

        '''updating the grid states accounting for the predefined number of pawns in each color -4 '''
        grid_states_red_ensured = ensure_minimum_keys_true(grid_states_red,grid_histories_red)
        grid_states_blue_ensured = ensure_minimum_keys_true(grid_states_blue,grid_histories_blue)
        grid_states_green_ensured = ensure_minimum_keys_true(grid_states_green,grid_histories_green)
        grid_states_yellow_ensured = ensure_minimum_keys_true(grid_states_yellow,grid_histories_yellow)

        '''updated the historeies of the grid tiles'''
        grid_histories_red = update_history(grid_histories_red,grid_states_red)
        grid_histories_blue = update_history(grid_histories_blue,grid_states_blue)
        grid_histories_green = update_history(grid_histories_green,grid_states_green)
        grid_histories_yellow = update_history(grid_histories_yellow,grid_states_yellow)

        '''updated the ensured historeis of the grid - accounting for the predefined number of pawns '''
        grid_histories_red_ensured = update_history(grid_histories_red,grid_states_red_ensured)
        grid_histories_blue_ensured = update_history(grid_histories_blue,grid_states_blue_ensured)
        grid_histories_green_ensured = update_history(grid_histories_green,grid_states_green_ensured)
        grid_histories_yellow_ensured = update_history(grid_histories_yellow,grid_states_yellow_ensured)


        
        '''checking if there was some change -pawns movement , if so the parameters for the message
        to be displayed are set '''
        if key_to_show_red and key_to_show_red != red_moved_to and 'red_base' not in key_to_show_red:
            red_moved_to = key_to_show_red
            pawn_movement = 'red pawn moved'
            if key_to_show_red == 'red_start':
                pawn_movement = 'red pawn entered the game'
            movement_color = (0, 0, 255)
        elif key_to_show_blue and key_to_show_blue != blue_moved_to and 'blue_base' not in key_to_show_blue:
            blue_moved_to = key_to_show_blue
            pawn_movement = 'blue pawn moved'
            if key_to_show_blue == 'blue_start':
                pawn_movement = 'blue pawn entered the game'
            movement_color = (255, 0, 0)
        elif key_to_show_yellow and key_to_show_yellow != yellow_moved_to and 'yellow_base' not in key_to_show_yellow:
            yellow_moved_to = key_to_show_yellow
            pawn_movement = 'yellow pawn moved'
            if key_to_show_yellow == 'yellow_start':
                pawn_movement = 'yellow pawn entered the game'
            movement_color = (0, 255, 255)
        elif key_to_show_green and key_to_show_green != green_moved_to and 'green_base' not in key_to_show_green:
            green_moved_to = key_to_show_green
            pawn_movement = 'green pawn moved'
            if key_to_show_green == 'green_start':
                pawn_movement = 'green pawn entered the game'
            movement_color = (0, 255, 0)
        
        cv2.putText(game_control_frame, pawn_movement, (190,225), cv2.FONT_HERSHEY_SIMPLEX, 0.5, movement_color, 2)
        



        
        

        '''the detected position of the pawns are annotated on the grid'''
        for grid_id, is_occupied in grid_states_red_ensured.items():
            if is_occupied:
                centroid = get_centroid(scaled_annotations[grid_id][0])
                cv2.putText(game_control_frame, "R", centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        grid_states_red = reset_detections(grid_states_red)

        for grid_id, is_occupied in grid_states_blue_ensured.items():
            if is_occupied:
                centroid = get_centroid(scaled_annotations[grid_id][0])
                cv2.putText(game_control_frame, "B", centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        grid_states_blue = reset_detections(grid_states_blue)

        for grid_id, is_occupied in grid_states_green_ensured.items():
            if is_occupied:
                centroid = get_centroid(scaled_annotations[grid_id][0])
                cv2.putText(game_control_frame, "G", centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        grid_states_green = reset_detections(grid_states_green)

        for grid_id, is_occupied in grid_states_yellow_ensured.items():
            if is_occupied:
                centroid = get_centroid(scaled_annotations[grid_id][0])
                cv2.putText(game_control_frame, "Y", centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        grid_states_yellow = reset_detections(grid_states_yellow)

        

        '''the dice is deteceted '''
        frame_annotated, tracker, tracking, stable_count, dice_pips, dice_img, prev_bbox, calculated = detect_dice_with_pips(
            frame_3, frame_annotated, mask_binary_white, tracker, tracking, frame_counter, stable_count, dice_pips, prev_bbox, calculated
        )
        if dice_pips is not None:
            last_dice_pips = dice_pips
            cv2.putText(game_control_frame, f'dice rolled: {last_dice_pips } rolled', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        else:
            cv2.putText(game_control_frame, f'dice rolled: {last_dice_pips } rolled', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255, 255, 255) , 2)

        '''the state of the game is displayed'''
        cv2.imshow("inscribed frame",game_control_frame)

    '''the  frame must be periodically alligned  to account for an kind of shaking, grid movement camera movement
    done every 100 frame to balance the qualitative  performance and the  computational complexity'''
    if frame_counter % 200 == 0:
        current_annotations = allign_frame(frame_3, reference_pick)



    

    '''the annotated frame is displayed '''
    cv2.imshow("annotated frame", frame_annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_counter += 1

cv2.destroyAllWindows()




